In [6]:
# importing libraries

import numpy as np
import pandas as pd
import psycopg2 as pg2
import nltk
import re
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [33]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [34]:
# defining functions

def getDataFromDB(query):
    
    try:
        conn = pg2.connect(dbname='twitterDB',user='postgres',password='postgres',host='localhost',port='5432')
        
        if conn.closed ==0:  # checks the system connection is active
            print("Connected to the DB")
            
            cur = conn.cursor()
            
            cur.execute(query)
            records = cur.fetchall()
            
            print("Data fetched")
            
            return records
            
    except Exception as e:
        print("Error thrown")
        print(e)
    
    finally:
        print("Closing connection")
        cur.close()
        conn.close()
        
    

In [35]:
query1 = 'select * from tweets'

In [36]:
val = getDataFromDB(query1)

Connected to the DB
Data fetched
Closing connection


In [37]:
df = pd.DataFrame(val,columns=['index','username','time_created','tweet','retweet_count','place_tweet','location_user'])

In [38]:
df.head()

,index,username,time_created,tweet,retweet_count,place_tweet,location_user
0,3,lionslide,2019-01-15 19:12:47+05:30,RT @iKNOMOMMY: LOOK AT LOVE SCENARIO SITTING T...,0,None,None
1,4,octopusjokes,2019-01-15 19:12:47+05:30,"Take a bow, Ms. Channing",0,the library,None
2,5,hawley_lisa,2019-01-15 19:12:47+05:30,RT @ScottMStedman: It would be a shame if this...,0,Germany (Not Really),None
3,6,SoniaSofia80,2019-01-15 19:12:47+05:30,RT @m_yosry2012: Coming baby 😂 https://t.co/Rx...,0,North,None
4,7,dominicdreams,2019-01-15 19:12:47+05:30,@ShadowhuntersTV Why they can be like this all...,0,spain,None
